In [ ]:
!pip install kaggle pillow pillow-heif

In [ ]:
import os
import json
import shutil
from pathlib import Path
import zipfile
from tqdm.notebook import tqdm

In [ ]:
# Setup Kaggle API credentials
os.makedirs("/root/.kaggle", exist_ok=True)
kaggle_token = {
    "username": "<your-username>",
    "key": "<your-kaggle-api-key>"
}

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_token, f)
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [ ]:
# Download dataset from Kaggle
api = KaggleApi()
api.authenticate()

dataset_dir = Path("./data/CATI-FAS_dataset")
if not dataset_dir.exists():
    dataset_dir.mkdir(parents=True)
    api.dataset_download_files(
        "n24q02m/cati-fas-face-anti-spoofing-dataset", path=str(dataset_dir), unzip=True
    )
    print("Downloading dataset...")
else:
    print("Dataset already exists.")

In [ ]:
# Rename files in live and spoof folders
def rename_files(directory):
    files = list(Path(directory).rglob("*.*"))
    for idx, file in enumerate(tqdm(files, desc=f"Renaming files in {directory}")):
        new_name = f"{idx+1:06d}{file.suffix}"
        new_path = file.parent / new_name
        file.rename(new_path)


rename_files(dataset_dir / "live")
rename_files(dataset_dir / "spoof")

In [ ]:
# Upload updated dataset to Kaggle
metadata = {
    "title": "CATI-FAS - Face Anti-Spoofing Dataset",
    "id": "n24q02m/cati-fas-face-anti-spoofing-dataset",
    "licenses": [{"name": "CC0-1.0"}],
}
metadata_path = dataset_dir / "dataset-metadata.json"
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

api = KaggleApi()
api.authenticate()
print("Uploading dataset to Kaggle...")
api.dataset_create_new(folder=str(dataset_dir), dir_mode="zip", quiet=False)
print("Dataset created successfully on Kaggle!")